In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe


class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper, epsilon=0.6):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            # 1 - 0.6/pi = 0.8090140682897256
            threshold = 1- epsilon/np.pi,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


### BERT-based-uncased

In [7]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8095684803001876
RF 0.725140712945591
KNN 0.6801125703564728
SVC 0.8030018761726079


In [9]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [10]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [11]:
runAttack(bert_model_wrapper_lr_mr, -1, 'attack_results_new/bertmrattacklr06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklr06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:46<2:44:26,  9.30s/it]textattack: Saving checkpoint under "checkpoints\1720023633646.ta.chkpt" at 2024-07-04 00:20:33 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [01:02<1:49:56,  6.25s/it]textattack: Saving checkpoint under "checkpoints\1720023649616.ta.chkpt" at 2024-07-04 00:20:49 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:55<2:14:54,  7.70s/it]textattack: Saving checkpoint under "checkpoints\1720023702684.ta.chkpt" at 2024-07-04 00:21:42 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 845 / 10 / 200 / 1055:  99%|█████████████▊| 1055/1066 [1:14:41<00:46,  4.25s/it]textattack: Saving checkpoint under "checkpoints\1720028068192.ta.chkpt" at 2024-07-04 01:34:28 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 845 / 10 / 201 / 1056:  99%|█████████████▊| 1056/1066 [1:14:41<00:42,  4.24s/it]

[Succeeded / Failed / Skipped / Total] 849 / 10 / 201 / 1060:  99%|█████████████▉| 1060/1066 [1:15:06<00:25,  4.25s/it]textattack: Saving checkpoint under "checkpoints\1720028093177.ta.chkpt" at 2024-07-04 01:34:53 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 852 / 10 / 203 / 1065: 100%|█████████████▉| 1065/1066 [1:15:41<00:04,  4.26s/it]textattack: Saving checkpoint under "checkpoints\1720028128486.ta.chkpt" at 2024-07-04 01:35:28 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 853 / 10 / 203 / 1066: 100%|██████████████| 1066/1066 [1:15:44<00:00,  4.26s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 853    |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 203    |
| Original accuracy:            | 80.96% |
| Accuracy under attack:        | 0.94%  |
| Attack success rate:          | 98.84% |
| Average perturbed word %:     | 15.1%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 81.16  |
+-------------------------------+--------+


In [12]:
runAttack(bert_model_wrapper_rf_mr, -1, 'attack_results_new/bertmrattackrf06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrf06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:27<1:37:28,  5.51s/it]textattack: Saving checkpoint under "checkpoints\1720028159390.ta.chkpt" at 2024-07-04 01:35:59 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [00:27<1:21:42,  4.62s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [00:42<1:15:11,  4.27s/it]textattack: Saving checkpoint under "checkpoints\1720028174552.ta.chkpt" at 2024-07-04 01:36:14 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 0 / 5 / 15:   1%|▎                     | 15/1066 [01:02<1:12:43,  4.15s/it]textattack: Saving checkpoint under "checkpoints\1720028194106.ta.chkpt" at 2024-07-04 01:36:34 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 741 / 24 / 290 / 1055:  99%|█████████████▊| 1055/1066 [1:08:55<00:43,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1720032267698.ta.chkpt" at 2024-07-04 02:44:27 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 746 / 24 / 290 / 1060:  99%|█████████████▉| 1060/1066 [1:09:30<00:23,  3.93s/it]textattack: Saving checkpoint under "checkpoints\1720032302404.ta.chkpt" at 2024-07-04 02:45:02 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 748 / 24 / 293 / 1065: 100%|█████████████▉| 1065/1066 [1:09:39<00:03,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1720032311801.ta.chkpt" at 2024-07-04 02:45:11 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 749 / 24 / 293 / 1066: 100%|██████████████| 1066/1066 [1:09:40<00:00,  3.92s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 749    |
| Number of failed attacks:     | 24     |
| Number of skipped attacks:    | 293    |
| Original accuracy:            | 72.51% |
| Accuracy under attack:        | 2.25%  |
| Attack success rate:          | 96.9%  |
| Average perturbed word %:     | 13.82% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 82.27  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_knn_mr, -1, 'attack_results_new/bertmrattackknn06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknn06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   0%|                         | 5/1066 [01:18<4:36:34, 15.64s/it]textattack: Saving checkpoint under "checkpoints\1720032391080.ta.chkpt" at 2024-07-04 02:46:31 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▏                        | 6/1066 [01:18<3:50:40, 13.06s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [01:26<2:31:29,  8.61s/it]textattack: Saving checkpoint under "checkpoints\1720032398955.ta.chkpt" at 2024-07-04 02:46:38 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   1%|▎                      | 15/1066 [02:43<3:10:22, 10.87s/it]textattack: Saving checkpoint under "checkpoints\1720032475900.ta.chkpt" at 2024-07-04 02:47:55 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 710 / 8 / 337 / 1055:  99%|████████████████▊| 1055/1066 [42:58<00:26,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1720034891694.ta.chkpt" at 2024-07-04 03:28:11 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 714 / 8 / 338 / 1060:  99%|████████████████▉| 1060/1066 [43:14<00:14,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1720034907599.ta.chkpt" at 2024-07-04 03:28:27 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 717 / 8 / 340 / 1065: 100%|████████████████▉| 1065/1066 [43:23<00:02,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1720034916825.ta.chkpt" at 2024-07-04 03:28:36 after 1065 attacks.
[Succeeded / Failed / Skipped / Total] 717 / 8 / 340 / 1065: 100%|█████████████████| 1066/1066 [43:24<00:00,  2.44s/it]

[Succeeded / Failed / Skipped / Total] 717 / 8 / 341 / 1066: 100%|█████████████████| 1066/1066 [43:24<00:00,  2.44s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 717    |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 341    |
| Original accuracy:            | 68.01% |
| Accuracy under attack:        | 0.75%  |
| Attack success rate:          | 98.9%  |
| Average perturbed word %:     | 8.88%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 53.51  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_svc_mr, -1, 'attack_results_new/bertmrattacksvc06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvc06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:31<1:53:06,  6.40s/it]textattack: Saving checkpoint under "checkpoints\1720034949221.ta.chkpt" at 2024-07-04 03:29:09 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                        | 6/1066 [00:32<1:34:35,  5.35s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   1%|▏                      | 10/1066 [00:47<1:24:07,  4.78s/it]textattack: Saving checkpoint under "checkpoints\1720034965033.ta.chkpt" at 2024-07-04 03:29:25 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:52<2:11:41,  7.52s/it]textattack: Saving checkpoint under "checkpoints\1720035030006.ta.chkpt" at 2024-07-04 03:30:30 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 835 / 13 / 207 / 1055:  99%|█████████████▊| 1055/1066 [1:14:37<00:46,  4.24s/it]textattack: Saving checkpoint under "checkpoints\1720039395123.ta.chkpt" at 2024-07-04 04:43:15 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 839 / 13 / 208 / 1060:  99%|█████████████▉| 1060/1066 [1:15:02<00:25,  4.25s/it]textattack: Saving checkpoint under "checkpoints\1720039420049.ta.chkpt" at 2024-07-04 04:43:40 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 842 / 13 / 210 / 1065: 100%|█████████████▉| 1065/1066 [1:15:38<00:04,  4.26s/it]textattack: Saving checkpoint under "checkpoints\1720039455661.ta.chkpt" at 2024-07-04 04:44:15 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 843 / 13 / 210 / 1066: 100%|██████████████| 1066/1066 [1:15:41<00:00,  4.26s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 843    |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 210    |
| Original accuracy:            | 80.3%  |
| Accuracy under attack:        | 1.22%  |
| Attack success rate:          | 98.48% |
| Average perturbed word %:     | 14.91% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 79.99  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [15]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [17]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [18]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [19]:
runAttack(disbert_model_wrapper_lr_mr, -1, 'attack_results_new/disbertmrattacklr06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklr06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:33<1:56:54,  6.61s/it]textattack: Saving checkpoint under "checkpoints\1720039721479.ta.chkpt" at 2024-07-04 04:48:41 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 7/1066 [00:33<1:23:44,  4.74s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   1%|▏                      | 10/1066 [00:40<1:11:14,  4.05s/it]textattack: Saving checkpoint under "checkpoints\1720039728904.ta.chkpt" at 2024-07-04 04:48:48 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:03<1:14:21,  4.25s/it]textattack: Saving checkpoint under "checkpoints\1720039752100.ta.chkpt" at 2024-07-04 04:49:12 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 829 / 9 / 217 / 1055:  99%|████████████████▊| 1055/1066 [38:45<00:24,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1720042013484.ta.chkpt" at 2024-07-04 05:26:53 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 829 / 9 / 218 / 1056:  99%|████████████████▊| 1056/1066 [38:45<00:22,  2.20s/it]

[Succeeded / Failed / Skipped / Total] 833 / 9 / 218 / 1060:  99%|████████████████▉| 1060/1066 [39:00<00:13,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1720042029028.ta.chkpt" at 2024-07-04 05:27:09 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 837 / 9 / 219 / 1065: 100%|████████████████▉| 1065/1066 [39:16<00:02,  2.21s/it]textattack: Saving checkpoint under "checkpoints\1720042044968.ta.chkpt" at 2024-07-04 05:27:24 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 838 / 9 / 219 / 1066: 100%|█████████████████| 1066/1066 [39:18<00:00,  2.21s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 838    |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 219    |
| Original accuracy:            | 79.46% |
| Accuracy under attack:        | 0.84%  |
| Attack success rate:          | 98.94% |
| Average perturbed word %:     | 15.47% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 79.6   |
+-------------------------------+--------+


In [20]:
runAttack(disbert_model_wrapper_rf_mr, -1, 'attack_results_new/disbertmrattackrf06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrf06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                        | 5/1066 [03:28<12:17:40, 41.72s/it]textattack: Saving checkpoint under "checkpoints\1720042255235.ta.chkpt" at 2024-07-04 05:30:55 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▏                       | 6/1066 [03:28<10:14:17, 34.77s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                      | 10/1066 [03:39<6:27:11, 22.00s/it]textattack: Saving checkpoint under "checkpoints\1720042266649.ta.chkpt" at 2024-07-04 05:31:06 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                     | 15/1066 [04:12<4:54:37, 16.82s/it]textattack: Saving checkpoint under "checkpoints\1720042298962.ta.chkpt" at 2024-07-04 05:31:38 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 767 / 42 / 246 / 1055:  99%|███████████████▊| 1055/1066 [49:54<00:31,  2.84s/it]textattack: Saving checkpoint under "checkpoints\1720045040948.ta.chkpt" at 2024-07-04 06:17:20 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 771 / 43 / 246 / 1060:  99%|███████████████▉| 1060/1066 [50:15<00:17,  2.84s/it]textattack: Saving checkpoint under "checkpoints\1720045062314.ta.chkpt" at 2024-07-04 06:17:42 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 775 / 43 / 247 / 1065: 100%|███████████████▉| 1065/1066 [50:36<00:02,  2.85s/it]textattack: Saving checkpoint under "checkpoints\1720045083617.ta.chkpt" at 2024-07-04 06:18:03 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 776 / 43 / 247 / 1066: 100%|████████████████| 1066/1066 [50:38<00:00,  2.85s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 776    |
| Number of failed attacks:     | 43     |
| Number of skipped attacks:    | 247    |
| Original accuracy:            | 76.83% |
| Accuracy under attack:        | 4.03%  |
| Attack success rate:          | 94.75% |
| Average perturbed word %:     | 16.44% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 95.83  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_knn_mr, -1, 'attack_results_new/disbertmrattackknn06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknn06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:29<1:42:45,  5.81s/it]textattack: Saving checkpoint under "checkpoints\1720045114201.ta.chkpt" at 2024-07-04 06:18:34 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▏                        | 7/1066 [00:29<1:13:44,  4.18s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                      | 10/1066 [00:45<1:19:37,  4.52s/it]textattack: Saving checkpoint under "checkpoints\1720045130396.ta.chkpt" at 2024-07-04 06:18:50 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                     | 15/1066 [00:59<1:09:11,  3.95s/it]textattack: Saving checkpoint under "checkpoints\1720045144394.ta.chkpt" at 2024-07-04 06:19:04 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 717 / 53 / 285 / 1055:  99%|███████████████▊| 1055/1066 [39:56<00:24,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720047481434.ta.chkpt" at 2024-07-04 06:58:01 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 721 / 53 / 286 / 1060:  99%|███████████████▉| 1060/1066 [40:11<00:13,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1720047497124.ta.chkpt" at 2024-07-04 06:58:17 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 724 / 53 / 288 / 1065: 100%|███████████████▉| 1065/1066 [40:22<00:02,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720047507183.ta.chkpt" at 2024-07-04 06:58:27 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 725 / 53 / 288 / 1066: 100%|████████████████| 1066/1066 [40:23<00:00,  2.27s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 725    |
| Number of failed attacks:     | 53     |
| Number of skipped attacks:    | 288    |
| Original accuracy:            | 72.98% |
| Accuracy under attack:        | 4.97%  |
| Attack success rate:          | 93.19% |
| Average perturbed word %:     | 10.73% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 83.34  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_svc_mr, -1, 'attack_results_new/disbertmrattacksvc06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvc06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   0%|                         | 5/1066 [00:31<1:52:10,  6.34s/it]textattack: Saving checkpoint under "checkpoints\1720047540146.ta.chkpt" at 2024-07-04 06:59:00 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▏                        | 7/1066 [00:31<1:20:19,  4.55s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   1%|▏                      | 10/1066 [00:40<1:10:55,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1720047548724.ta.chkpt" at 2024-07-04 06:59:08 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   1%|▎                     | 15/1066 [01:03<1:13:46,  4.21s/it]textattack: Saving checkpoint under "checkpoints\1720047571619.ta.chkpt" at 2024-07-04 06:59:31 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 819 / 10 / 226 / 1055:  99%|███████████████▊| 1055/1066 [37:58<00:23,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1720049787137.ta.chkpt" at 2024-07-04 07:36:27 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 819 / 10 / 227 / 1056:  99%|███████████████▊| 1056/1066 [37:58<00:21,  2.16s/it]

[Succeeded / Failed / Skipped / Total] 823 / 10 / 227 / 1060:  99%|███████████████▉| 1060/1066 [38:14<00:12,  2.16s/it]textattack: Saving checkpoint under "checkpoints\1720049802669.ta.chkpt" at 2024-07-04 07:36:42 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 827 / 10 / 228 / 1065: 100%|███████████████▉| 1065/1066 [38:30<00:02,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1720049819376.ta.chkpt" at 2024-07-04 07:36:59 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 828 / 10 / 228 / 1066: 100%|████████████████| 1066/1066 [38:32<00:00,  2.17s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 828    |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 228    |
| Original accuracy:            | 78.61% |
| Accuracy under attack:        | 0.94%  |
| Attack success rate:          | 98.81% |
| Average perturbed word %:     | 15.59% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 80.7   |
+-------------------------------+--------+


### CLIP

In [23]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [24]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [25]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [26]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [27]:
runAttack(clip_model_wrapper_lr_mr, -1, 'attack_results_new/clipmrattacklr06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklr06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:39<2:19:39,  7.90s/it]textattack: Saving checkpoint under "checkpoints\1720050558020.ta.chkpt" at 2024-07-04 07:49:18 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                        | 6/1066 [00:39<1:56:38,  6.60s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   1%|▏                      | 10/1066 [00:50<1:28:38,  5.04s/it]textattack: Saving checkpoint under "checkpoints\1720050568884.ta.chkpt" at 2024-07-04 07:49:28 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:   1%|▎                     | 15/1066 [01:17<1:30:39,  5.18s/it]textattack: Saving checkpoint under "checkpoints\1720050596145.ta.chkpt" at 2024-07-04 07:49:56 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 787 / 3 / 265 / 1055:  99%|████████████████▊| 1055/1066 [30:58<00:19,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1720052376582.ta.chkpt" at 2024-07-04 08:19:36 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 787 / 3 / 265 / 1055:  99%|████████████████▊| 1056/1066 [30:58<00:17,  1.76s/it]

[Succeeded / Failed / Skipped / Total] 791 / 3 / 266 / 1060:  99%|████████████████▉| 1060/1066 [31:05<00:10,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1720052384247.ta.chkpt" at 2024-07-04 08:19:44 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 795 / 3 / 267 / 1065: 100%|████████████████▉| 1065/1066 [31:19<00:01,  1.76s/it]textattack: Saving checkpoint under "checkpoints\1720052397518.ta.chkpt" at 2024-07-04 08:19:57 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 796 / 3 / 267 / 1066: 100%|█████████████████| 1066/1066 [31:19<00:00,  1.76s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 796    |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 267    |
| Original accuracy:            | 74.95% |
| Accuracy under attack:        | 0.28%  |
| Attack success rate:          | 99.62% |
| Average perturbed word %:     | 13.01% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 65.45  |
+-------------------------------+--------+


In [28]:
runAttack(clip_model_wrapper_rf_mr, -1, 'attack_results_new/clipmrattackrf06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrf06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [01:43<6:07:23, 20.78s/it]textattack: Saving checkpoint under "checkpoints\1720052502619.ta.chkpt" at 2024-07-04 08:21:42 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                        | 6/1066 [01:44<5:06:17, 17.34s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                      | 10/1066 [02:00<3:32:03, 12.05s/it]textattack: Saving checkpoint under "checkpoints\1720052519229.ta.chkpt" at 2024-07-04 08:21:59 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   1%|▎                     | 15/1066 [02:19<2:42:48,  9.29s/it]textattack: Saving checkpoint under "checkpoints\1720052538163.ta.chkpt" at 2024-07-04 08:22:18 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 715 / 28 / 312 / 1055:  99%|███████████████▊| 1055/1066 [39:48<00:24,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1720054787119.ta.chkpt" at 2024-07-04 08:59:47 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 719 / 28 / 313 / 1060:  99%|███████████████▉| 1060/1066 [39:56<00:13,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1720054795387.ta.chkpt" at 2024-07-04 08:59:55 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 723 / 28 / 314 / 1065: 100%|███████████████▉| 1065/1066 [40:08<00:02,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1720054807005.ta.chkpt" at 2024-07-04 09:00:07 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 724 / 28 / 314 / 1066: 100%|████████████████| 1066/1066 [40:09<00:00,  2.26s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 724    |
| Number of failed attacks:     | 28     |
| Number of skipped attacks:    | 314    |
| Original accuracy:            | 70.54% |
| Accuracy under attack:        | 2.63%  |
| Attack success rate:          | 96.28% |
| Average perturbed word %:     | 13.18% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 78.44  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_knn_mr, -1, 'attack_results_new/clipmrattackknn06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknn06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   0%|                         | 5/1066 [01:53<6:42:40, 22.77s/it]textattack: Saving checkpoint under "checkpoints\1720054921752.ta.chkpt" at 2024-07-04 09:02:01 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   1%|▏                      | 10/1066 [01:59<3:30:56, 11.99s/it]textattack: Saving checkpoint under "checkpoints\1720054927768.ta.chkpt" at 2024-07-04 09:02:07 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   1%|▎                     | 15/1066 [02:22<2:46:45,  9.52s/it]textattack: Saving checkpoint under "checkpoints\1720054950689.ta.chkpt" at 2024-07-04 09:02:30 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 674 / 50 / 331 / 1055:  99%|███████████████▊| 1055/1066 [41:09<00:25,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1720057277341.ta.chkpt" at 2024-07-04 09:41:17 after 1055 attacks.


[Succeeded / Failed / Skipped / Total] 679 / 50 / 331 / 1060:  99%|███████████████▉| 1060/1066 [41:17<00:14,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1720057285472.ta.chkpt" at 2024-07-04 09:41:25 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 683 / 50 / 332 / 1065: 100%|███████████████▉| 1065/1066 [41:26<00:02,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1720057293907.ta.chkpt" at 2024-07-04 09:41:33 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 684 / 50 / 332 / 1066: 100%|████████████████| 1066/1066 [41:26<00:00,  2.33s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 684    |
| Number of failed attacks:     | 50     |
| Number of skipped attacks:    | 332    |
| Original accuracy:            | 68.86% |
| Accuracy under attack:        | 4.69%  |
| Attack success rate:          | 93.19% |
| Average perturbed word %:     | 9.97%  |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 78.51  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_svc_mr, -1, 'attack_results_new/clipmrattacksvc06.csv', target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvc06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   0%|                         | 5/1066 [00:28<1:40:36,  5.69s/it]textattack: Saving checkpoint under "checkpoints\1720057323291.ta.chkpt" at 2024-07-04 09:42:03 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▏                        | 6/1066 [00:28<1:24:05,  4.76s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   1%|▏                      | 10/1066 [00:35<1:03:17,  3.60s/it]textattack: Saving checkpoint under "checkpoints\1720057330808.ta.chkpt" at 2024-07-04 09:42:10 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   1%|▎                     | 15/1066 [01:00<1:11:06,  4.06s/it]textattack: Saving checkpoint under "checkpoints\1720057355738.ta.chkpt" at 2024-07-04 09:42:35 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 790 / 4 / 261 / 1055:  99%|████████████████▊| 1055/1066 [27:18<00:17,  1.55s/it]textattack: Saving checkpoint under "checkpoints\1720058933284.ta.chkpt" at 2024-07-04 10:08:53 after 1055 attacks.
[Succeeded / Failed / Skipped / Total] 790 / 4 / 262 / 1056:  99%|████████████████▊| 1056/1066 [27:18<00:15,  1.55s/it]

[Succeeded / Failed / Skipped / Total] 794 / 4 / 262 / 1060:  99%|████████████████▉| 1060/1066 [27:24<00:09,  1.55s/it]textattack: Saving checkpoint under "checkpoints\1720058939176.ta.chkpt" at 2024-07-04 10:08:59 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 798 / 4 / 263 / 1065: 100%|████████████████▉| 1065/1066 [27:35<00:01,  1.55s/it]textattack: Saving checkpoint under "checkpoints\1720058950044.ta.chkpt" at 2024-07-04 10:09:10 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 799 / 4 / 263 / 1066: 100%|█████████████████| 1066/1066 [27:35<00:00,  1.55s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 799    |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 263    |
| Original accuracy:            | 75.33% |
| Accuracy under attack:        | 0.38%  |
| Attack success rate:          | 99.5%  |
| Average perturbed word %:     | 12.82% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 65.04  |
+-------------------------------+--------+
